## 심장마비 확률이 높은사람?
- 성별, 나이, 혈압, 콜레스테롤, 공복혈당, 최대 심박수 등의 컬럼이 있음
- 평가: ROC-AUC, 정확도(Accuracy), F1 을 구하시오
- target : output (1:심장마비 확률 높음, 0:심장마비 확률 낮음)
- csv파일 생성 : 수험번호.csv (예시 아래 참조)
~~~
id,output
41,0.633
28,0.123
222,0.355
~~~



## 데이터 불러오기

In [ ]:
import pandas as pd

train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/main/p2/heart/2files/train.csv")
test =  pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/main/p2/heart/2files/test.csv")
# train = pd.read_csv("train.csv")
# test =  pd.read_csv("test.csv")

##EDA

In [ ]:
train.head()

,id,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,48,53,0,2,128,216,0,0,115,0,0.0,2,0,0,1
1,124,39,0,2,94,199,0,1,179,0,0.0,2,0,2,1
2,129,74,0,1,120,269,0,0,121,1,0.2,2,1,2,1
3,78,52,1,1,128,205,1,1,184,0,0.0,2,0,2,1
4,151,71,0,0,112,149,0,1,125,0,1.6,1,0,2,1


In [ ]:
test.head()

,id,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,41,48,1,1,130,245,0,0,180,0,0.2,1,0,2
1,28,65,0,2,140,417,1,0,157,0,0.8,2,1,2
2,222,65,1,3,138,282,1,0,174,0,1.4,1,1,2
3,106,69,1,3,160,234,1,0,131,0,0.1,1,1,2
4,108,50,0,1,120,244,0,1,162,0,1.1,2,0,2


In [ ]:
print(train.shape , test.shape)

(242, 15) (61, 14)


In [ ]:
train['output'].value_counts()

,count
output,
1,126
0,116


In [ ]:
train.isnull().sum()

,0
id,0
age,0
sex,0
cp,0
trtbps,0
chol,0
fbs,0
restecg,0
thalachh,0
exng,0


In [ ]:
test.isnull().sum()

,0
id,0
age,0
sex,0
cp,0
trtbps,0
chol,0
fbs,0
restecg,0
thalachh,0
exng,0


In [ ]:
train.describe()

,id,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
count,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.00000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000
mean,154.706612,54.516529,0.673554,0.913223,131.070248,246.933884,0.119835,0.53719,149.590909,0.347107,1.019835,1.380165,0.785124,2.347107,0.520661
std,86.328576,9.021513,0.469885,1.020897,17.716978,52.236414,0.325441,0.53183,21.763201,0.477037,1.182137,0.621285,1.052218,0.607160,0.500608
min,0.000000,34.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.00000,88.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,81.250000,47.250000,0.000000,0.000000,120.000000,212.000000,0.000000,0.00000,133.250000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,158.500000,55.000000,1.000000,1.000000,130.000000,243.000000,0.000000,1.00000,152.000000,0.000000,0.600000,1.000000,0.000000,2.000000,1.000000
75%,228.750000,61.000000,1.000000,2.000000,140.000000,274.000000,0.000000,1.00000,165.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,302.000000,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.00000,195.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        242 non-null    int64  
 1   age       242 non-null    int64  
 2   sex       242 non-null    int64  
 3   cp        242 non-null    int64  
 4   trtbps    242 non-null    int64  
 5   chol      242 non-null    int64  
 6   fbs       242 non-null    int64  
 7   restecg   242 non-null    int64  
 8   thalachh  242 non-null    int64  
 9   exng      242 non-null    int64  
 10  oldpeak   242 non-null    float64
 11  slp       242 non-null    int64  
 12  caa       242 non-null    int64  
 13  thall     242 non-null    int64  
 14  output    242 non-null    int64  
dtypes: float64(1), int64(14)
memory usage: 28.5 KB


In [ ]:
train.nunique()

#이걸 실행해서 결과값을 봤을 때, 성별이 2개, CP가4개 이런게 존재함으로 아마 레이블 인코딩이 된 케이스가 아닐까 라고 추측이 가능함
#왜냐면 숫자가 작은 것들이 다는것은, 분류가 되었고, 또 성별은 남,여만 존재하니까 뭔가 잘 되었다 라고 추측은 가능

,0
id,242
age,40
sex,2
cp,4
trtbps,47
chol,139
fbs,2
restecg,3
thalachh,83
exng,2


## 데이터 전처리 & 피처엔지니어링

In [ ]:
#다 숫자화 되어있어서 최소화 하는 작업을 해야함
#drop을 해서 최소화 하자
# ID 값 -> 결과에 전혀 영향을 미치지 않는 것 drop but 테스트 데이터에 있는거는 나중에 결과값 아이디와 inpput이므로 test데이터 값은 날리면 안됨.

train = train.drop('id', axis= 1)
test_id = test.pop('id')

In [ ]:
test.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,48,1,1,130,245,0,0,180,0,0.2,1,0,2
1,65,0,2,140,417,1,0,157,0,0.8,2,1,2
2,65,1,3,138,282,1,0,174,0,1.4,1,1,2
3,69,1,3,160,234,1,0,131,0,0.1,1,1,2
4,50,0,1,120,244,0,1,162,0,1.1,2,0,2


## 검증 데이터 분리

In [ ]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train.drop('output', axis=1), train['output'],
                                            test_size =0.15, random_state=2022)

#X 데이터 에는 output 드랍하고, y데이터에는 아웃풋만 넣으라는 거임
#내가 틀린 부분 axis 앞에 , 아닌 . 으로 씀 / random_state 밑에 언더바 안넣고 함

In [ ]:
# 검증 데이터 분리 후 꼭 head 찍어서 분리가 잘 되었는지 봐라
X_tr.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
43,53,0,0,130,264,0,0,143,0,0.4,1,0,2
32,55,0,0,128,205,0,2,130,1,2.0,1,1,3
190,61,1,3,134,234,0,1,145,0,2.6,1,2,2
211,38,1,2,138,175,0,1,173,0,0.0,2,4,2
234,64,1,3,170,227,0,0,155,0,0.6,1,0,3


## 모델 & 평가

In [ ]:
#2가지 모델을 만들어서 평가 ㄱ
#1. 랜포
#2. xgb




In [ ]:
#랜덤포레스트 = rf
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
rf = RandomForestClassifier(random_state=2022, max_depth=5, n_estimators=200)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
pred_proba = rf.predict_proba(X_val)
#두번쨰 줄 random_state=2022 꼭 괄호안에 넣어주어야함, 이거 안하면 반복할때마다 결과값이 달라져서 성능이 올라가기도 하고 떨어지기도 하기 떄문
#pred까지 선코딩후 에러가 발생하는 지 찍어보고  그다음에 다시 from으로 넘어가서 평가지표 코딩 이어 하기 이어서 진행
#평가 값은 실제값 y인 y_val과 우리가 예측한 값인 pred를 작성
#roc_auc하는 경우 predict가 아닌 predict_proba를 사용하기 때문에 꼭꼭 pred 및에 하나 더 써주고 예측값도 다른 걸로 써줘야함!
#proba의 경우 전체 데이터 중에서 인덱스 1번인 데이터만 평가로 넣기 [:,1] 처음 : 이전체, 뒤에가 1번
print(roc_auc_score(y_val, pred_proba[:,1]))
print(f1_score(y_val, pred))
print(accuracy_score(y_val, pred))

# ※랜포에서 하이터파라미터 튜닝시 크게 2가지
# 1. max_depth : 맥스뎁스가 없으면 트리의 깊이가 무한정 깊어 질수 있는것
                # 가이드 : 깊이는 3 ~ 12 까지만 바꿔가면서 해봐라 저 범위 외의 숫자 ㄴㄴ
                # 보통 3,5,7
                # -> 그냥 rf 값보다 rf max_depth=3 한것의 f1과 accuracy가 많이 올라간 것을 볼 수 있음
                # -> max_depth의 깊이 즉 숫자가 커질수록 예측값이 좋아지는것(올라가는것)은 아님  depth 5가 7보다 높음
                #-> 3과 5를 비교해보면 roc는 5가 좋은데 반면 f1이냐 accurcy는 3이 좋음 그래서 평가기준이 무엇이냐에 따라 어떤 걸로 할지 결정을 해야함


# 2. n_estinators -> 트리의 개수를 몇개를 만드냐는 것
                # 가이드 : 개수는 100 ~1000사이 값으로 바꿔가면서 해봐라, but 100미만으로는 가면 안됨!
                          # 보통 100, 200 ,400, 700, 1000
                          # -> but 개수를 많이 만들수록 즉 숫자가 커질수록 속도는 느려질 수 밖에 없음
                # -> 기본 값은 100 인데 -> 200으로 바꿀 경우 -> roc_auc는 더 올라감



#밑에 모델을 활용해서 평가를 하고, 하이퍼파라미터 튜닝을 해서 성능을 올린뒤 가장 높은 값으로 결정! 이중에서는 #rf max_depth=5 , n_estimators=200 이 가장 높았음


#rf
# 0.9301242236024845
# 0.7906976744186046
# 0.7567567567567568

#rf max_depth=3
# 0.9378881987577639
# 0.8636363636363636
# 0.8378378378378378

#rf max_depth=5
# 0.9409937888198757
# 0.8444444444444444
# 0.8108108108108109

#rf max_depth=7
# 0.9347826086956521
# 0.8444444444444444
# 0.8108108108108109

#rf max_depth=5 , n_estimators=200
# 0.9440993788819876
# 0.8444444444444444
# 0.810810810810810


#rf max_depth=5 , n_estimators=400
# 0.9409937888198757
# 0.8444444444444444
# 0.8108108108108109


0.9440993788819876
0.8444444444444444
0.8108108108108109


In [ ]:
#xgb
#learning_rate는 부스팅 계열 알고리즘에서만 사용가능! -> 학습속도를 제어
#랜포에서는 사용 불가 -> 이유 각 트리가 독립적으로 학습되기 때문에 learning_rat같은 속도 제어 개념 불필요
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state=2022, max_depth =5, n_estimators = 600, learning_rate=0.01)
xgb.fit(X_tr, y_tr)
pred = xgb.predict(X_val)
pred_proba = xgb.predict_proba(X_val)

print(roc_auc_score(y_val, pred_proba[:,1]))
print(f1_score(y_val, pred))
print(accuracy_score(y_val, pred))

#하이퍼파라미터 튜닝을 통해 점수를 얼마나 높여갈 수 있는지 연습


# xgb에서 하이터파라미터 튜닝
# 1. max_depth : 기본 3       / 비교 : 3 4 5 6 7 8 9 12
# 2. n_estimators = 기본 100  / 비교 : 100 ~1000
# 3. learning_rate = 기본 0.1 / 비교 : 0.1 ~0.01

#estimators 가 올라갈때 learning_rate값을 낮춰 주어야함!!!!! 같이 올리면 절대 안됨!!!

#xgb는 약한 학습기임 rf에 비해서 그래서 learning_rate를 해주면 좋다




#나중에 할때는 올라가는 것만 놓고 비교하면 됨. 그러면 최적의 것을 찾을 수 있으니까!
# 안좋은 모델은 굳이 기록할 필요 없음 어차피 안쓸거라

#xgb
# 0.9192546583850931
# 0.8444444444444444
# 0.8108108108108109


#xgb, max_depth =3
# 0.8975155279503105
# 0.8444444444444444
# 0.8108108108108109

#xgb, max_depth =5
# 0.906832298136646
# 0.8181818181818182
# 0.7837837837837838

#xgb, max_depth =7
# 0.9099378881987576
# 0.8181818181818182
# 0.7837837837837838


#xgb, max_depth =5 ,  n_estimators = 200
# 0.9068322981366459
# 0.8181818181818182
# 0.7837837837837838



#xgb, max_depth =5 ,  n_estimators = 200 learning_rate=0.05
# 0.9037267080745341
# 0.8181818181818182
# 0.7837837837837838


#xgb, max_depth =5 ,  n_estimators = 400 learning_rate=0.01
# 0.9161490683229814
# 0.8372093023255814
# 0.8108108108108109

#xgb, max_depth =5 ,  n_estimators = 600 learning_rate=0.01

# 0.9161490683229814
# 0.8372093023255814
# 0.8108108108108109






0.9161490683229814
0.8372093023255814
0.8108108108108109


## 예측 및 csv 제출

In [ ]:
test.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,48,1,1,130,245,0,0,180,0,0.2,1,0,2
1,65,0,2,140,417,1,0,157,0,0.8,2,1,2
2,65,1,3,138,282,1,0,174,0,1.4,1,1,2
3,69,1,3,160,234,1,0,131,0,0.1,1,1,2
4,50,0,1,120,244,0,1,162,0,1.1,2,0,2


In [ ]:
# 데이터 프레임 만들기전 맨위 문제를 한번 더 읽어보고, 컬럼이랑, 숫자인지, 확률값인지 등등 봐라

In [ ]:
pred_proba = xgb.predict_proba(test)

In [ ]:
#CSV 파일 만들기
pd.DataFrame({'id':test_id, 'output':pred_proba[:,1]}).to_csv("0000.csv", index=False)

In [ ]:
#데이터가 정상적으로 불러와 지는지 화인
pd.read_csv("0000.csv")

,id,output
0,41,0.978896
1,28,0.889302
2,222,0.658906
3,106,0.309241
4,108,0.973777
...,...,...
56,6,0.951077
57,269,0.018985
58,2,0.989059
59,14,0.967539


##정리

In [ ]:
#성적예측

y_test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/main/p2/heart/2files/y_test.csv")
print(roc_auc_score(y_test, pred_proba[:,1]))

0.8694638694638694
